In [1]:
from datetime import datetime
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np 
import pandas as pd

Отфильтруем время на дату, день и час

In [2]:
def filter_time(datefield):
    """Столбец делится на дату, день, час и добавляются в виде отдельных столбцов"""
    i = 0
    for timestamp in datefield['datetime']:
        date_object = datetime.strptime(timestamp.split()[0], '%Y-%m-%d')
        hour = timestamp.split()[1][:2]
        # 0 - понедельник, 1 - вторник и т.д
        day = datetime.date(date_object).weekday()
        dict_year = {2011:1, 2012:2}
        year = dict_year[date_object.year]
        datefield.loc[i, 'day'] = day
        datefield.loc[i, 'hour'] = hour
        datefield.loc[i, 'year'] = year
        i += 1
    return datefield

Считаем данные и отфильтруем по времени

In [11]:
bike_dataframe = pd.read_csv("/home/emin/PycharmProjects/machine_learning/data/bike_sharing/train.csv")
bike_dataframe = filter_time(bike_dataframe)
bike_dataframe = bike_dataframe[['datetime', 'year', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count']]
bike_dataframe.head()

,datetime,year,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1.0,5.0,00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1.0,5.0,01,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1.0,5.0,02,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1.0,5.0,03,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1.0,5.0,04,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [9]:
column = ['year', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']
bike_dataframe[column][2:-1]

,year,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
2,1.0,5.0,02,1,0,0,1,9.02,13.635,80,0.0000
3,1.0,5.0,03,1,0,0,1,9.84,14.395,75,0.0000
4,1.0,5.0,04,1,0,0,1,9.84,14.395,75,0.0000
5,1.0,5.0,05,1,0,0,2,9.84,12.880,75,6.0032
6,1.0,5.0,06,1,0,0,1,9.02,13.635,80,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
10880,2.0,2.0,18,4,0,1,1,15.58,19.695,50,23.9994
10881,2.0,2.0,19,4,0,1,1,15.58,19.695,50,26.0027
10882,2.0,2.0,20,4,0,1,1,14.76,17.425,57,15.0013
10883,2.0,2.0,21,4,0,1,1,13.94,15.910,61,15.0013


In [13]:
test_dataframe = pd.read_csv("/home/emin/PycharmProjects/machine_learning/data/bike_sharing/test.csv")
test_dataframe = filter_time(test_dataframe)
test_dataframe = test_dataframe[['datetime', 'year', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']]
test_dataframe.head()

,datetime,year,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1.0,3.0,00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1.0,3.0,01,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1.0,3.0,02,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1.0,3.0,03,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1.0,3.0,04,1,0,1,1,10.66,12.880,56,11.0014


Создаем случайный лес с 500 деревьями, при 3 объектах происходит расщепление и оцениваем

In [18]:
random_forest_object = RandomForestRegressor(n_estimators=500, min_samples_split=3, oob_score=True)
# для случайных велосипедистов
collumn_fit = ['year', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']
# перекрестная проверка случайных велосипедистов
results = cross_val_score(random_forest_object, bike_dataframe[collumn_fit], bike_dataframe['casual'], cv=5)
print(results)

[0.55961343 0.80127506 0.27426555 0.70395589 0.82810036]


Отчёт о точности работы

In [19]:
print("Точность: %0.2f (+/- %0.2f)" % (results.mean(), results.std()*2))

Точность: 0.63 (+/- 0.41)


Обучаем случайный лес

In [20]:
casual_obj = random_forest_object.fit(bike_dataframe[collumn_fit], bike_dataframe['casual'] )
casual_obj.feature_importances_

array([0.03674468, 0.06096532, 0.35293798, 0.01709758, 0.00301424,
       0.14169272, 0.00976209, 0.07486832, 0.22443349, 0.05726153,
       0.02122205])

Предсказываем

In [21]:
test_dataframe['casual'] = random_forest_object.predict(test_dataframe[collumn_fit])
test_dataframe.head()

,datetime,year,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual
0,2011-01-20 00:00:00,1.0,3.0,00,1,0,1,1,10.66,11.365,56,26.0027,2.114590
1,2011-01-20 01:00:00,1.0,3.0,01,1,0,1,1,10.66,13.635,56,0.0000,0.735467
2,2011-01-20 02:00:00,1.0,3.0,02,1,0,1,1,10.66,13.635,56,0.0000,1.299833
3,2011-01-20 03:00:00,1.0,3.0,03,1,0,1,1,10.66,12.880,56,11.0014,0.696467
4,2011-01-20 04:00:00,1.0,3.0,04,1,0,1,1,10.66,12.880,56,11.0014,0.447733


Обучаем модель на зарегистрированных велосипедистов

In [23]:
# для зарегестрированных велосипедистов
regullar_collumn = ['year', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']
# перекрестная проверка зарегестрированных велосипедистов
results = cross_val_score(random_forest_object, bike_dataframe[regullar_collumn], bike_dataframe['registered'], cv=5)
print(results)

[0.06992555 0.88434542 0.72908286 0.85545252 0.87170113]


Отчёт о точности работы

In [24]:
print("Точность: %0.2f (+/- %0.2f)" % (results.mean(), results.std()*2))

Точность: 0.68 (+/- 0.62)


In [25]:
registered_obj = random_forest_object.fit(bike_dataframe[regullar_collumn], bike_dataframe['registered'] )
registered_obj.feature_importances_

array([0.09246417, 0.02475065, 0.5785747 , 0.05514078, 0.00102087,
       0.14113701, 0.0155085 , 0.02980708, 0.02423004, 0.02791571,
       0.0094505 ])

Прогнозы на тестовом наборе данных

In [26]:
test_dataframe['registered'] = random_forest_object.predict(test_dataframe[regullar_collumn])
test_dataframe['count'] = test_dataframe['casual'] + test_dataframe['registered']
test_dataframe.head()

,datetime,year,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-20 00:00:00,1.0,3.0,00,1,0,1,1,10.66,11.365,56,26.0027,2.114590,10.429288,12.543879
1,2011-01-20 01:00:00,1.0,3.0,01,1,0,1,1,10.66,13.635,56,0.0000,0.735467,4.901102,5.636568
2,2011-01-20 02:00:00,1.0,3.0,02,1,0,1,1,10.66,13.635,56,0.0000,1.299833,1.959652,3.259486
3,2011-01-20 03:00:00,1.0,3.0,03,1,0,1,1,10.66,12.880,56,11.0014,0.696467,1.797500,2.493967
4,2011-01-20 04:00:00,1.0,3.0,04,1,0,1,1,10.66,12.880,56,11.0014,0.447733,1.769383,2.217117


Округляем количество до целого числа

In [28]:
test_dataframe['count'] = test_dataframe['count'].apply(np.round)
test_dataframe.head()

,datetime,year,day,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-20 00:00:00,1.0,3.0,00,1,0,1,1,10.66,11.365,56,26.0027,2.114590,10.429288,13.0
1,2011-01-20 01:00:00,1.0,3.0,01,1,0,1,1,10.66,13.635,56,0.0000,0.735467,4.901102,6.0
2,2011-01-20 02:00:00,1.0,3.0,02,1,0,1,1,10.66,13.635,56,0.0000,1.299833,1.959652,3.0
3,2011-01-20 03:00:00,1.0,3.0,03,1,0,1,1,10.66,12.880,56,11.0014,0.696467,1.797500,2.0
4,2011-01-20 04:00:00,1.0,3.0,04,1,0,1,1,10.66,12.880,56,11.0014,0.447733,1.769383,2.0


Вывод результата в csv таблицу

In [29]:
# генерим результаты в submission.csv
dataframe_csv = test_dataframe[['datetime', 'count']]
dataframe_csv.to_csv('/home/emin/PycharmProjects/machine_learning/data/bike_sharing/submission.csv', index=False)
dataframe_csv.head()

,datetime,count
0,2011-01-20 00:00:00,13.0
1,2011-01-20 01:00:00,6.0
2,2011-01-20 02:00:00,3.0
3,2011-01-20 03:00:00,2.0
4,2011-01-20 04:00:00,2.0
